In [ ]:
import pandas as pd 
import numpy as np

inputPing_Start = 64209

amountTested = 50

inputPing_End = inputPing_Start + amountTested

testInput_Start = inputPing_End
testInput_End = testInput_Start + amountTested




df_stave = pd.read_parquet('real_numbers_STAVE.parquet')

df_binned = pd.read_csv('3000.csv')

df_input = df_stave[df_stave['pingNum'] < inputPing_End]
df_input = df_input[df_input['pingNum'] >= inputPing_Start]
df_input = df_input[df_input['channel']%2 == 0]
df_input = df_input[['sample']]
df_input = df_input.reset_index()
df_input = df_input.drop(['index'], axis=1)



df_output = df_binned[df_binned['PingKey'] < inputPing_End]
df_output = df_output[df_output['PingKey'] >= inputPing_Start]
df_output = df_output[df_output['Channel'] == 0]
df_output = df_output.reset_index()
df_output = df_output.drop(['index'], axis=1)
df_output = df_output[['Angle', 'TWTT']]
df_output_angle = df_output[['Angle']]
df_output_twtt = df_output[['TWTT']]



df_input_test = df_stave[df_stave['pingNum'] < testInput_End]
df_input_test = df_input_test[df_input_test['pingNum'] >= testInput_Start]
df_input_test = df_input_test[df_input_test['channel']%2 == 0]
df_input_test = df_input_test[['sample']]
df_input_test = df_input_test.reset_index()
df_input_test = df_input_test.drop(['index'], axis=1)



df_output_test = df_binned[df_binned['PingKey'] < testInput_End]
df_output_test = df_output_test[df_output_test['PingKey'] >= testInput_Start]
df_output_test = df_output_test[df_output_test['Channel'] == 0]
df_output_test = df_output_test.reset_index()
df_output_test = df_output_test.drop(['index'], axis=1)
df_output_test = df_output_test[['Angle', 'TWTT']]
df_output_test_angle = df_output_test[['Angle']]
df_output_test_twtt = df_output_test[['TWTT']]



In [ ]:
inputSize = df_input.shape[0]
print(inputSize)

outputSize = df_output.shape[0]
print(outputSize)

df_input = np.asarray(df_input)
df_input = df_input.reshape(1,inputSize )
print(df_input.shape)


df_output_angle = np.asarray(df_output_angle)
df_output_twtt = np.asarray(df_output_test_twtt)
df_out_angle = df_output_angle.reshape(1,outputSize)
df_out_twtt = df_output_twtt.reshape(1,outputSize)


output_test_angle = np.asarray(df_output_test_angle)
output_test_twtt = np.asarray(df_output_test_twtt)
# df_output_test_twtt = np.asarray(df_output_test_twtt)
# df_output_test_angle = np.asarray(df_output_test_angle)
# df_out_test_angle = df_output_test_angle.reshape(1,4000)
# df_out_test_twtt = df_output_test_twtt.reshape(1,4000)

df_input_test = np.asarray(df_input_test)
df_input_test = df_input_test.reshape(1,inputSize  )
print(df_input_test.shape)





In [ ]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense


In [ ]:
inputs = Input(shape=(inputSize,))
# hidden_Angle0 =  Dense(1, kernel_initializer='lecun_normal', activation='selu')(inputs)
# hidden_Angle1 = Dense(512, kernel_initializer='lecun_normal', activation='selu')(hidden_Angle0)
hidden_Angle2 = Dense(256, kernel_initializer='lecun_normal', activation='selu')(inputs)
hidden_Angle3 = Dense(128, kernel_initializer='lecun_normal', activation='selu')(hidden_Angle2)
hidden_Angle4 = Dense(64, kernel_initializer='lecun_normal', activation='selu')(hidden_Angle3)
output_Angle = Dense(outputSize, activation='linear', name = 'Angles')(hidden_Angle4)

hidden_twtt1 = Dense(400, activation = 'softmax')(inputs)
hidden_twtt2 = Dense(300, activation = 'softmax')(hidden_twtt1)
hidden_twtt3 = Dense(200, activation = 'softmax')(hidden_twtt2)
# I lowered the number of neurons for the TWTT and it does well
output_twtt = Dense(outputSize, activation='linear', name = 'TWTT')(hidden_twtt3)
# hidden_twtt1 = 

model = Model(inputs=inputs, outputs=[output_Angle, output_twtt])



In [ ]:

model.compile(optimizer='adamax', loss='mse', metrics=['mean_squared_error', 'accuracy'])

model.fit(df_input, [df_out_angle,df_out_twtt], epochs=400, batch_size=40, validation_data=(df_input_test, [output_test_angle,output_test_twtt]))

In [ ]:
prediction_angle, prediction_twtt = model.predict(df_input_test)
prediction_angle = prediction_angle.reshape(outputSize)
prediction_twtt = prediction_twtt.reshape(outputSize)


df = pd.DataFrame({'Angle': prediction_angle, 'TWTT': prediction_twtt})

df 

# df_out_test = pd.DataFrame({'Angle_raw': df_output_test_angle, 'TWTT_raw': df_output_test_twtt})


In [ ]:
# df_out_test_angle = df_out_test_angle.reshape(1,4000)
# df_out_test_twtt = df_out_test_twtt.reshape(1,4000)

df_output_test_angle = df_output_test[['Angle']]
df_output_test_twtt = df_output_test[['TWTT']]

df['Angle_Raw'] = df_output_test_angle
df['TWTT_Raw'] = df_output_test_twtt



df

In [ ]:
df_twtt = df[['TWTT', 'TWTT_Raw']]
df_angle = df[['Angle','Angle_Raw']]
df['diff'] = (df['TWTT_Raw'] - df['TWTT'])
df['diff'] = df['diff'].abs()
df['diff'] = df['diff']/df['TWTT_Raw']
df['diff'] = df['diff']*100
avg = df['diff'].mean()
print(100 - avg.round(2))
twtt_image = df_twtt.plot.line()
print(twtt_image)
# df_twtt

In [ ]:
import matplotlib.pyplot as plt 

df_angle['diff'] = (df_angle['Angle_Raw'] - df_angle['Angle'])
df_angle['diff'] = df_angle['diff'].abs()
df_angle['diff'] = df_angle['diff']/df_angle['Angle_Raw']
df_angle['diff'] = df_angle['diff']*100
avg = df_angle['diff'].mean()
print(100 - avg.round(2))
df_angle['index'] = df_angle.index
plt.scatter(df_angle['index'], df_angle['Angle_Raw'], color = 'black', s=.5)
plt.scatter(df_angle['index'], df_angle['Angle'], color = 'red', s=.5)
plt.legend(loc='upper left')

# angle_image = df_angle.plot.line()




